In [1]:
import pandas as pd
from morfeusz2 import Morfeusz
morfeusz = Morfeusz() 

In [2]:
pd.set_option('display.max_colwidth', None)

## Loading data and choosing only appropiate MWE

In [3]:
poleval_df = pd.read_csv("poleval2019_task2_training_190221/index.tsv", sep="\t", header=None)
poleval_df.columns = ["phrase_id", "doc_id", "text", "lemma"]
poleval_df.head()

,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
1,40025,99883,Toronto,Toronto
2,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
3,31833,99883,Joe Fafard,Joe Fafard
4,327365,99883,kanadyjskim,kanadyjski


In [4]:
print(f"Shape of the original Poleval 2019 dataset: {poleval_df.shape}")

Shape of the original Poleval 2019 dataset: (22177, 4)


In [5]:
mwe_df = poleval_df[poleval_df["text"].str.split().str.len() > 1]
mwe_df.head()

,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
2,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
3,31833,99883,Joe Fafard,Joe Fafard
5,343872,99883,Ludwiga Mies van der Rohe,Ludwig Mies van der Rohe
7,343871,99883,Toronto Dominion Gallery of Inuit Art,Toronto Dominion Gallery of Inuit Art


In [6]:
print(f"Shape of the Poleval 2019 dataset (only multi word expressions): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (only multi word expressions): (9718, 4)


**remove not polish MWE**


In [7]:
def is_polish_mwe(text):
    """
    Check if the text is a valid Polish MWE.
    Returns True if at least one word is recognized (not tagged as 'ign').
    """
    analysis = morfeusz.analyse(text)
    for _, _, interpretations in analysis:
        tag = interpretations[2]
        if tag != 'ign':
            return True
    return False

In [8]:
df_unrecognized = mwe_df[~mwe_df['text'].apply(is_polish_mwe)].reset_index(drop=True)
print(f"Number of unrecognized MWEs: {len(df_unrecognized)}")
df_unrecognized.sample(10)

Number of unrecognized MWEs: 566


,phrase_id,doc_id,text,lemma
16,99721,100524,Barvicha Luxury Village,Barvicha Luxury Village
393,287881,102435,History Carnival,History Carnival
482,471925,103954,Angry Birds,Angry Birds
225,72068,101751,Creative Commons,Creative Commons
473,471465,103927,Andrei Strbik,Andrea Strbik
161,67462,101384,Tiengiz Grigorjewicz Sułakwielidze,Tiengiz Grigorjewicz Sułakwielidze
549,490467,107326,Nicolasem Sarkozy,Nicolas Sarkozy
63,52562,101186,Pearson Television,Pearson Television
271,73836,101886,Two Harbors,Two Harbors
57,52395,101173,Olle Hansson,Olle Hansson


In [9]:
mwe_df = mwe_df[mwe_df["text"].apply(is_polish_mwe)].reset_index(drop=True)
print(f"Shape of the Poleval 2019 dataset (only valid Polish MWEs): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (only valid Polish MWEs): (9152, 4)


There was 566 fully not recognized expressions by Morfeusz.


**Remove duplicates**


In [10]:
def remove_duplicates(df):
    """
    Remove duplicates from the DataFrame based on 'text' and 'lemma'.
    Keeps the first occurrence.
    """
    return df.drop_duplicates(subset=["text", "lemma"], keep='first')

In [11]:
mwe_df = remove_duplicates(mwe_df)
print(f"Shape of the Poleval 2019 dataset (after removing duplicates): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (after removing duplicates): (7453, 4)


**TO DO: podzielić na nieodmienne i mianowniki:**


In [12]:
mwe_df[mwe_df["text"] == mwe_df["lemma"]]


,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
1,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
2,31833,99883,Joe Fafard,Joe Fafard
4,343871,99883,Toronto Dominion Gallery of Inuit Art,Toronto Dominion Gallery of Inuit Art
5,343875,100499,PZL.13 (PZL-13),PZL.13 (PZL-13)
...,...,...,...,...
9137,329445,107360,Marin Leovac,Marin Leovac
9139,329446,107360,Roland Linz,Roland Linz
9142,329448,107360,Tomas Jun,Tomas Jun
9144,329449,107360,Patrick Salomon,Patrick Salomon


In [13]:
mwe_df = mwe_df[mwe_df["text"] != mwe_df["lemma"]]


In [14]:
print(f"Shape of the Poleval 2019 dataset (only inflected mwe): {mwe_df.shape}")


Shape of the Poleval 2019 dataset (only inflected mwe): (3828, 4)


In [15]:
mwe_df[['text', 'lemma']].sample(10)

,text,lemma
2283,Lexusa IS,Lexus IS
280,igrzyskach w Nagano,igrzyska w Nagano
3188,Sesji Sejmu Dzieci i Młodzieży,Sejm Dzieci i Młodzieży
7531,A Johnsona,A Johnson
1673,Anny Marii z domu Petardówny,Anna Maria z domu Petardówny
7322,stanów kwantowych,stany kwantowe
1746,Willa Wrighta,Will Wright
2100,Osiedle XXV-lecia PRL,osiedle XXV-lecia PRL
6803,Porozumienia Międzyinstytucjonalnego,Porozumienie Międzyinstytucjonalne
1137,departamencie Loiret,departament Loiret


## Cleaning

In [16]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        1
dtype: int64

In [17]:
mwe_df[mwe_df["lemma"].isna()]

,phrase_id,doc_id,text,lemma
1334,307628,101286,San Diego Rockets,NaN


In [ ]:
# I drop it because it's not a polish MWE
mwe_df = mwe_df.dropna()

In [19]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        0
dtype: int64

In [20]:
mwe_df[mwe_df["text"] == "Dolce & Gabbana"]

,phrase_id,doc_id,text,lemma
161,99725,100524,Dolce & Gabbana,Dolce &amp; Gabbana


In [21]:
mwe_df[mwe_df["text"] == "Lublin R-VIII a"]

,phrase_id,doc_id,text,lemma
734,65875,101200,Lublin R-VIII a,Lublin R-VIII


In [22]:
mwe_df[mwe_df["text"] == "BBC Radio 1's Live Lounge"]

,phrase_id,doc_id,text,lemma
2264,426217,101434,BBC Radio 1's Live Lounge,BBC Radio 1


In [23]:
mwe_df[mwe_df["text"] == "Jammin' CRT.5"]

,phrase_id,doc_id,text,lemma
5108,468926,102125,Jammin' CRT.5,Jammin


In [24]:
mwe_df[mwe_df["text"] == "Traxxas Rustler XL-1 '09"]


,phrase_id,doc_id,text,lemma
6580,469441,102506,Traxxas Rustler XL-1 '09,Traxxas Rustler XL-1


In [25]:
mwe_df[mwe_df["text"] == "Żerań F S O"]


,phrase_id,doc_id,text,lemma
6916,350246,102673,Żerań F S O,Żerań FSO


In [26]:
mwe_df.loc[mwe_df["text"] == "Żerań F S O", "text"] = "Żerań FSO"

In [27]:
mwe_df.loc[mwe_df["text"] == "R C S Orzeł", "text"] = "RCS Orzeł"

In [28]:
mwe_df = mwe_df[mwe_df["text"] != "S K M"]
mwe_df = mwe_df[mwe_df["text"] != "R K S"]

In [29]:
mwe_df[mwe_df["lemma"].str.contains("ZTM")]

,phrase_id,doc_id,text,lemma
6897,352573,102669,Z T M,ZTM


In [30]:
mwe_df[mwe_df["lemma"].str.contains("WKD")]


,phrase_id,doc_id,text,lemma
6988,352545,102697,W K D,WKD


In [31]:
mwe_df[mwe_df["text"] == "átlátszó ("]

,phrase_id,doc_id,text,lemma
8097,481079,103921,átlátszó (,átlátszó


In [32]:
mwe_df[mwe_df["text"] == "Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”"]

,phrase_id,doc_id,text,lemma
5224,79818,102157,Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”,Społeczna Rada Konsultacyjna ds. aktualizacji„Strategii Rozwoju Bydgoszczy do 2015 roku


In [33]:
mwe_df.loc[mwe_df["text"] == 'Społecznej Rady Konsultacyjnej ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku”', "lemma"] = 'Społeczna Rada Konsultacyjna ds. aktualizacji „Strategii Rozwoju Bydgoszczy do 2015 roku"'

In [34]:
mwe_df[mwe_df["text"].str.contains("Fundacji Ośrodka")]

,phrase_id,doc_id,text,lemma
6812,71729,102602,"Fundacji Ośrodka "" Karta ""","Fundacja Ośrodka ""Karta """


In [35]:
mwe_df.loc[mwe_df["text"] == 'Fundacji Ośrodka " Karta "', ["text", "lemma"]] = ['Fundacji Ośrodka "Karta"', 'Fundacja Ośrodka "Karta"']

In [36]:
mwe_df[mwe_df["text"].str.contains("FILLER")] # error during creating dataset

,phrase_id,doc_id,text,lemma
6973,474289,102694,Dworca FILLER Centralnego,Dworzec Centralny
6978,350524,102696,Dworzec FILLER Śródmieście,Dworzec Śródmieście


In [37]:
mwe_df.loc[mwe_df["text"] == 'Dworca FILLER Centralnego', "text"] = 'Dworca Centralnego'
mwe_df.loc[mwe_df["text"] == 'Dworzec FILLER Śródmieście', "text"] = 'Dworzec Śródmieście'

In [38]:
mwe_df[mwe_df["text"].str.contains("Herburt")]

,phrase_id,doc_id,text,lemma
1202,50764,101263,Jan Herburt (Arłamowski,Jan Herburt (Arłamowski)


In [39]:
mwe_df[mwe_df["text"].str.contains("Naukę")]

,phrase_id,doc_id,text,lemma
4075,310468,101854,Naukę 2.0,Nauka2.0


In [40]:
mwe_df.loc[mwe_df["text"] == "Naukę 2.0", "lemma"] = "Nauka 2.0"


In [41]:
mwe_df[mwe_df["text"] == "Sesji Sejmu Dzieci i Młodzieży"]

,phrase_id,doc_id,text,lemma
3188,63764,101638,Sesji Sejmu Dzieci i Młodzieży,Sejm Dzieci i Młodzieży


In [42]:
mwe_df.loc[mwe_df["text"] == "Sesji Sejmu Dzieci i Młodzieży", "lemma"] = "Sesja Sejmu Dzieci i Młodzieży"

In [43]:
mwe_df[mwe_df["text"]=='ustawy o zmianie ustawy Kodeks postępowania karnego']

,phrase_id,doc_id,text,lemma
3279,63911,101644,ustawy o zmianie ustawy Kodeks postępowania karnego,Ustawao zmianie ustawy Kodeks postępowania karnego


In [44]:
mwe_df.loc[mwe_df["text"]=='ustawy o zmianie ustawy Kodeks postępowania karnego', "lemma"] = 'ustawa o zmianie ustawy Kodeks postępowania karnego'


In [45]:
mwe_df[mwe_df["text"].str.contains("o ujawnieniu pracy lub służby")]


,phrase_id,doc_id,text,lemma
4449,81044,101946,o zmianie ustawy o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944–1990 osób pełniących funkcje publiczne,ustawa o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944–1990 osób pełniących funkcje publiczne
4451,81046,101946,ustawie z dnia 11 kwietnia 1997 r. o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944-1990 osób pełniących funkcje publiczne,Ustawa z dnia 11 kwietnia 1997 r. o ujawnieniu pracy lub służby w organach bezpieczeństwa państwa lub współpracy z nimi w latach 1944-1990 osób pełniących funkcje publiczne


In [46]:
mwe_df = mwe_df.drop(index=4449)

In [47]:
mwe_df.drop(mwe_df[mwe_df["text"].isin([
    "Dolce & Gabbana", # not a polish MWE
    "Lublin R-VIII a", # ???
    "BBC Radio 1's Live Lounge", # not a polish MWE
    "Jammin' CRT.5", # not a polish MWE
    "Traxxas Rustler XL-1 '09", # not a polish MWE
    "átlátszó (", # not a polish MWE
    "Z T M",
    "W K D"
])].index, inplace=True)

In [48]:
print(f"Shape of the Poleval 2019 dataset (after cleaning): {mwe_df.shape}")

Shape of the Poleval 2019 dataset (after cleaning): (3816, 4)


In [49]:
mwe_df.to_csv("mwe_cleaned_df.csv")

## Actual analysis

In [3]:
mwe_df = pd.read_csv("mwe_cleaned_df.csv", index_col=0)
print(f"Shape of the cleaned Poleval 2019 dataset: {mwe_df.shape}")

Shape of the cleaned Poleval 2019 dataset: (3816, 4)


In [4]:
mwe_df.head()

,phrase_id,doc_id,text,lemma
3,343872,99883,Ludwiga Mies van der Rohe,Ludwig Mies van der Rohe
7,343879,100499,samolotu PZL.23 „Karaś”,samolot PZL.23 „Karaś”
8,31843,100499,Ministerstwa Komunikacji,Ministerstwo Komunikacji
11,343880,100499,silnikiem Pratt-Whitney Wasp,silnik Pratt-Whitney Wasp
14,31839,100499,Stanisława Praussa,Stanisław Prauss


In [5]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        0
dtype: int64

In [6]:
def is_conjunction(word):
    """
    Check if the word is a conjunction.
    Returns True if the word is a conjunction.
    """
    analysis = morfeusz.analyse(word)
    for _, _, interpretations in analysis:
        tag = interpretations[2]
        if tag.startswith('conj'):
            return True
    return False

In [ ]:
def analyze_pair(words, lemmas):
    """
    Analyze a 2 word expression.
    :param words: List of 2 words in the expression.
    :param lemmas: List of 2 lemmas corresponding to the words.
    :return: number, case, relation type and head of the expression.
    """
    # possible numbers and cases for the words
    number_case = {}
    part_of_speech = {}

    # returned values
    number = None
    case = None
    relation = None
    head = None

    #print(f"Analyzing pair: {words} with lemmas: {lemmas}")

    for i, word in enumerate(words):
        number_case_i = set()
        analyses = morfeusz.analyse(word)

        for _, _, interpretations in analyses:
            tag = interpretations[2].split(":")
            if len(tag) > 2:
                number_i = tag[1]
                case_i = tag[2].split('.')

                # Morfeusz can return more than 1 possibilty of case after the dot so I make a list of it 
                for c in case_i:
                    number_case_i.add((number_i, c))

                part_of_speech[word] = tag[0]

        number_case[i] = number_case_i

    available = [i for i in number_case if number_case[i]]

    # check if it is possible to find common number and case for the words in the mwe -> ZWIĄZEK ZGODY
    common_cases = set.intersection(*number_case.values())
    if len(common_cases)>=1:
        # if there is only one common pair(number, case), then it is an agreement relation and the number and case of the expression can be assigned to the whole expression
        
        if len(common_cases) == 1:
            number_case_head_list = list(common_cases)[0]
            number = number_case_head_list[0]
            case = number_case_head_list[1]
 
        else:
            number = [x[0] for x in list(common_cases)]
            case = [x[1] for x in list(common_cases)]
            
        relation = "agreement"

        # noun -> HEAD
        for i, (word, tag) in enumerate(part_of_speech.items()):
            if tag == 'subst':
                head = lemmas[i]
                break

        # if there is no noun in the expression, then the head is the first word in the expression
            else:
                head = lemmas[0]

    elif [w_i for w_i, l_i in zip(words, lemmas) if w_i == l_i]:
        # government relation    
        for i, word in enumerate(words):
            if word == lemmas[i]:
                continue
            else: 
                if number_case[i]:
                    head = lemmas[i]
                    number_case_head_list = list(number_case[i])
                    number = [x[0] for x in number_case_head_list]
                    case = [x[1] for x in number_case_head_list]
                    relation = "government"
        
    if len(available) == 1:
        #print(f"Only one word has a valid analysis: {words[available[0]]} with lemma {lemmas[available[0]]}, words: {words}, lemmas: {lemmas}, number_case: {number_case}")
        # if there is only one word in the expression that has a valid analysis, then it is the head of the expression
        head = lemmas[available[0]]
        number_case_head_list = list(number_case[available[0]])
        number = [x[0] for x in number_case_head_list]
        case = [x[1] for x in number_case_head_list]
        
        idx_j = 1 - available[0]  # index of the second word in the pair
        if words[idx_j] == lemmas[idx_j]:
            # if the second word is the same as the lemma, then it is a government relation
            relation = "government"
        else:
            relation = "agreement"

    return number, case, relation, head


def analyze_mwe(phrase, lemma):
    """
    Analyze a multi-word expression.
    :param phrase: The multi-word expression.
    :param lemma: The lemma of the multi-word expression.
    :return: analysis results - possible: numbers, cases, relations, heads, pairs.
    """

    phrase = phrase.replace(" - ", "-")
    phrase = phrase.replace("___", "")

    
    words = phrase.strip().split()
    lemmas = lemma.strip().split() 

    numbers = []
    cases = []
    relations = []
    heads = []
    pairs = []

    i = 0
    while i < len(words)-1:
        if is_conjunction(words[i+1]) and i + 2 < len(words):
            pair_words = [words[i], words[i+2]]
            pair_lemmas = [lemmas[i], lemmas[i+2]]
            i += 2 # skip the conjunction
        
        else:
            pair_words = words[i:i+2]
            pair_lemmas = lemmas[i:i+2]
            i += 1 

        # analyze the pair of words
        analyses = analyze_pair(pair_words, pair_lemmas)
        number, case, relation, head = analyses
        pairs.append(tuple(pair_words))
        numbers.append(number)  
        cases.append(case)
        relations.append(relation)
        heads.append(head)


    return numbers, cases, relations, heads, pairs

In [8]:
mwe_df[["numbers", "cases", "relations", "heads", "pairs"]] = mwe_df.apply(lambda row: pd.Series(analyze_mwe(row["text"], row["lemma"])), axis=1)

In [32]:
mwe_df.sample(20)

,phrase_id,doc_id,text,lemma,numbers,cases,relations,heads,pairs
7307,468088,103621,atomów trójpoziomowych,atomy trójpoziomowe,[pl],[gen],[agreement],[atomy],"[(atomów, trójpoziomowych)]"
5200,468117,102155,twierdzenia Eulera,twierdzenie Eulera,[sg],[gen],[agreement],[twierdzenie],"[(twierdzenia, Eulera)]"
5388,467703,102205,biegłość żeglarską,biegłość żeglarska,[sg],[acc],[agreement],[biegłość],"[(biegłość, żeglarską)]"
421,40626,100626,Sokolnikach Lesie,Sokolniki Las,[None],[None],[None],[None],"[(Sokolnikach, Lesie)]"
932,352109,101232,armią Republiki Wietnamu,armia Republiki Wietnamu,"[[sg], sg]","[[inst], gen]","[government, agreement]","[armia, Republiki]","[(armią, Republiki), (Republiki, Wietnamu)]"
7876,484379,103861,E. Benveniste’a,E. Benveniste,"[[sg, sg]]","[[gen, acc]]",[government],[Benveniste],"[(E., Benveniste’a)]"
874,53788,101224,Danieli z Kleinbergerów,Daniela z Kleinbergerów,"[[sg, pl, sg, pl, sg, sg, sg, pl, sg], [gen, inst]]","[[nom, voc, loc, gen, voc, acc, gen, nom, dat], [nwok, nwok]]","[government, government]","[Daniela, z]","[(Danieli, z), (z, Kleinbergerów)]"
4421,78544,101943,Europejskiemu Inspektorowi Ochrony Danych,Europejski Inspektor Ochrony Danych,"[sg, [sg], pl]","[dat, [dat], acc]","[agreement, government, agreement]","[Inspektor, Inspektor, Ochrony]","[(Europejskiemu, Inspektorowi), (Inspektorowi, Ochrony), (Ochrony, Danych)]"
5921,484835,102378,bibliotekami cyfrowymi,biblioteki cyfrowe,[pl],[inst],[agreement],[biblioteki],"[(bibliotekami, cyfrowymi)]"
3753,472766,101745,centrum handlowym,centrum handlowe,"[[sg, sg]]","[[loc, inst]]",[agreement],[centrum],"[(centrum, handlowym)]"


In [34]:
mwe_df.sample(20)

,phrase_id,doc_id,text,lemma,numbers,cases,relations,heads,pairs
7084,466566,102756,Departamentowi Pożytku Publicznego,Departament Pożytku Publicznego,"[[pl, pl, sg], sg]","[[voc, nom, dat], gen]","[government, agreement]","[Departament, Pożytku]","[(Departamentowi, Pożytku), (Pożytku, Publicznego)]"
5814,337526,102353,Zbigniewa Ziobro,Zbigniew Ziobro,[sg],[acc],[agreement],[Zbigniew],"[(Zbigniewa, Ziobro)]"
6982,351874,102697,Warszawy Śródmieście,Warszawa Śródmieście,"[[pl, pl, sg, pl]]","[[voc, acc, gen, nom]]",[government],[Warszawa],"[(Warszawy, Śródmieście)]"
3516,467159,101678,premiera Donalda Tuska,premier Donald Tusk,"[[sg, sg], [sg, sg]]","[[gen, acc], [gen, acc]]","[agreement, agreement]","[premier, Donald]","[(premiera, Donalda), (Donalda, Tuska)]"
972,54168,101236,Bolesława III Krzywoustego,Bolesław III Krzywousty,"[[sg, sg, sg], [sg, sg]]","[[nom, gen, acc], [acc, gen]]","[government, government]","[Bolesław, Krzywousty]","[(Bolesława, III), (III, Krzywoustego)]"
8587,318908,107287,Wszystkich Świętych,Wszyscy Święci,"[[pl, pl, pl]]","[[loc, gen, acc]]",[agreement],[Wszyscy],"[(Wszystkich, Świętych)]"
948,308653,101234,Chrystusa Zbawiciela,Chrystus Zbawiciel,"[[sg, sg]]","[[gen, acc]]",[agreement],[Chrystus],"[(Chrystusa, Zbawiciela)]"
6987,351877,102697,Dworca Śródmieście,dworzec Śródmieście,[[sg]],[[gen]],[government],[dworzec],"[(Dworca, Śródmieście)]"
9035,327777,107354,Woli Mieleckiej,Wola Mielecka,"[[sg, sg, sg]]","[[dat, loc, gen]]",[agreement],[Wola],"[(Woli, Mieleckiej)]"
5366,467549,102181,Ignacym Wołodkowiczem,Ignacy Wołodkowicz,[sg],[inst],[agreement],[Ignacy],"[(Ignacym, Wołodkowiczem)]"


In [17]:
analyze_pair(["Objawienia",  "Pańskiego"], ["Objawienie", "Pański"])	

('sg', 'gen', 'agreement', 'Pański')

In [22]:
morfeusz.analyse("diecezji koszalińsko-kołobrzeskiej")

[(0, 1, ('diecezji', 'diecezja', 'subst:sg:gen:f', ['nazwa_pospolita'], [])),
 (0,
  1,
  ('diecezji', 'diecezja', 'subst:sg:dat.loc:f', ['nazwa_pospolita'], [])),
 (0,
  1,
  ('diecezji', 'diecezja', 'subst:pl:gen:f', ['nazwa_pospolita'], ['hom.'])),
 (1, 2, ('koszalińsko', 'koszaliński', 'adja', [], [])),
 (2, 3, ('-', '-', 'interp', [], [])),
 (3, 4, ('kołobrzeskiej', 'kołobrzeski', 'adj:sg:dat:f:pos', [], [])),
 (3, 4, ('kołobrzeskiej', 'kołobrzeski', 'adj:sg:gen:f:pos', [], [])),
 (3, 4, ('kołobrzeskiej', 'kołobrzeski', 'adj:sg:loc:f:pos', [], []))]